# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse  Shipments          Customer  \
0  Apr 20 2023  1:10PM  261513        10  MSP220968  Methapharm, Inc.   
1  Apr 20 2023  1:10PM  261513        10  MSP220986  Methapharm, Inc.   
2  Apr 20 2023  1:10PM  261513        10  MSP220987  Methapharm, Inc.   
3  Apr 20 2023  1:10PM  261513        10  MSP220988  Methapharm, Inc.   
4  Apr 20 2023  1:10PM  261513        10  MSP220990  Methapharm, Inc.   
5  Apr 20 2023  1:10PM  261513        10  MSP220991  Methapharm, Inc.   
6  Apr 20 2023  1:10PM  261513        10  MSP220992  Methapharm, Inc.   
7  Apr 20 2023  1:10PM  261513        10  MSP220993  Methapharm, Inc.   
8  Apr 20 2023  1:07PM  261512        10   HH-41434         Hush Hush   
9  Apr 20 2023  1:07PM  261512        10   HH-41435         Hush Hush   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
24  261509       Released          1
25  261510       Released          1
26  261511       Released          6
27  261512       Released          6
28  261513       Released          8

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
261509                NaN        NaN       1.0
261510                NaN        NaN       1.0
261511                NaN        NaN       6.0
261512                NaN        NaN       6.0
261513                NaN        NaN       8.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
261400                9.0       32.0       1.0
261432                0.0        0.0       1.0
261448                0.0        0.0       5.0
261452                0.0        0.0       1.0
261460                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
261400                  9         32         1
261432                  0          0         1
261448                  0          0         5
261452                  0          0         1
261460                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               261400          9         32         1
1               261432          0          0         1
2               261448          0          0         5
3               261452          0          0         1
4               261460          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               261400         9        32        1
1               261432                            1
2               261448                            5
3               261452                            1
4               261460                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                       Customer
0   Apr 20 2023  1:10PM  261513        10               Methapharm, Inc.
8   Apr 20 2023  1:07PM  261512        10                      Hush Hush
14  Apr 20 2023 12:46PM  261511        10                        Bio-PRF
20  Apr 20 2023 12:24PM  261510        10                        Bio-PRF
21  Apr 20 2023 12:13PM  261509        10  Nextsource Biotechnology, LLC
22  Apr 20 2023 11:56AM  261507        19            AdvaGen Pharma, Ltd
23  Apr 20 2023 11:35AM  261505        21              NBTY Global, Inc.
24  Apr 20 2023 11:23AM  261503        15    Virtus Pharmaceuticals, LLC
25  Apr 20 2023 11:22AM  261501        15          Alliance Pharma, Inc.
28  Apr 20 2023 11:17AM  261460        15           Person & Covey, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                       Customer  \
0  Apr 20 2023  1:10PM  261513        10               Methapharm, Inc.   
1  Apr 20 2023  1:07PM  261512        10                      Hush Hush   
2  Apr 20 2023 12:46PM  261511        10                        Bio-PRF   
3  Apr 20 2023 12:24PM  261510        10                        Bio-PRF   
4  Apr 20 2023 12:13PM  261509        10  Nextsource Biotechnology, LLC   
5  Apr 20 2023 11:56AM  261507        19            AdvaGen Pharma, Ltd   
6  Apr 20 2023 11:35AM  261505        21              NBTY Global, Inc.   
7  Apr 20 2023 11:23AM  261503        15    Virtus Pharmaceuticals, LLC   
8  Apr 20 2023 11:22AM  261501        15          Alliance Pharma, Inc.   
9  Apr 20 2023 11:17AM  261460        15           Person & Covey, Inc.   

  Completed Executing Released  
0                            8  
1                            6  
2                            6  
3                            1  
4                            1  
5                   1           
6                            1  
7                            1  
8                   1        2  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                       Customer  \
0  Apr 20 2023  1:10PM  261513        10               Methapharm, Inc.   
1  Apr 20 2023  1:07PM  261512        10                      Hush Hush   
2  Apr 20 2023 12:46PM  261511        10                        Bio-PRF   
3  Apr 20 2023 12:24PM  261510        10                        Bio-PRF   
4  Apr 20 2023 12:13PM  261509        10  Nextsource Biotechnology, LLC   
5  Apr 20 2023 11:56AM  261507        19            AdvaGen Pharma, Ltd   
6  Apr 20 2023 11:35AM  261505        21              NBTY Global, Inc.   
7  Apr 20 2023 11:23AM  261503        15    Virtus Pharmaceuticals, LLC   
8  Apr 20 2023 11:22AM  261501        15          Alliance Pharma, Inc.   
9  Apr 20 2023 11:17AM  261460        15           Person & Covey, Inc.   

  Released Executing Completed  
0        8                      
1        6                      
2        6                      
3        1                      
4        1                      
5                  1            
6        1                      
7        1                      
8        2         1            
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Apr 20 2023  1:10PM  261513        10               Methapharm, Inc.   
1  Apr 20 2023  1:07PM  261512        10                      Hush Hush   
2  Apr 20 2023 12:46PM  261511        10                        Bio-PRF   
3  Apr 20 2023 12:24PM  261510        10                        Bio-PRF   
4  Apr 20 2023 12:13PM  261509        10  Nextsource Biotechnology, LLC   

  Released Executing Completed  
0        8                      
1        6                      
2        6                      
3        1                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Apr 20 2023  1:10PM  261513        10               Methapharm, Inc.   
1  Apr 20 2023  1:07PM  261512        10                      Hush Hush   
2  Apr 20 2023 12:46PM  261511        10                        Bio-PRF   
3  Apr 20 2023 12:24PM  261510        10                        Bio-PRF   
4  Apr 20 2023 12:13PM  261509        10  Nextsource Biotechnology, LLC   

   Released  Executing  Completed  
0       8.0        NaN        NaN  
1       6.0        NaN        NaN  
2       6.0        NaN        NaN  
3       1.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Apr 20 2023  1:10PM  261513        10               Methapharm, Inc.   
1  Apr 20 2023  1:07PM  261512        10                      Hush Hush   
2  Apr 20 2023 12:46PM  261511        10                        Bio-PRF   
3  Apr 20 2023 12:24PM  261510        10                        Bio-PRF   
4  Apr 20 2023 12:13PM  261509        10  Nextsource Biotechnology, LLC   

   Released  Executing  Completed  
0       8.0        0.0        0.0  
1       6.0        0.0        0.0  
2       6.0        0.0        0.0  
3       1.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1307555      22.0        0.0        0.0
15         1830277      12.0       33.0        9.0
19         2091726      62.0       32.0        4.0
21          523005       1.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1307555      22.0        0.0        0.0
1        15  1830277      12.0       33.0        9.0
2        19  2091726      62.0       32.0        4.0
3        21   523005       1.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      22.0        0.0        0.0
1        15      12.0       33.0        9.0
2        19      62.0       32.0        4.0
3        21       1.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   22.0
1        15   Released   12.0
2        19   Released   62.0
3        21   Released    1.0
4        10  Executing    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15    19   21
Status                          
Completed   0.0   9.0   4.0  0.0
Executing   0.0  33.0  32.0  1.0
Released   22.0  12.0  62.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15    19   21
0          Completed   0.0   9.0   4.0  0.0
1          Executing   0.0  33.0  32.0  1.0
2           Released  22.0  12.0  62.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15    19   21
0  Completed   0.0   9.0   4.0  0.0
1  Executing   0.0  33.0  32.0  1.0
2   Released  22.0  12.0  62.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()